Import libraries

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import Trainer, TrainingArguments
from datasets import DatasetDict, load_dataset

c:\Users\cl502_19\Desktop\Domain-LLM\LLMProj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load Dataset and Model

In [2]:
dataset = load_dataset("json", data_files={
    "train": "mahasquad\\train.json",
    "validation": "mahasquad\\val.json",
    "test": "mahasquad\\test.json"
})

In [3]:
# Load the XLM-R tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/marathi-roberta")
model = AutoModelForQuestionAnswering.from_pretrained("l3cube-pune/marathi-roberta")

c:\Users\cl502_19\Desktop\Domain-LLM\LLMProj\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at l3cube-pune/marathi-roberta and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preprocessing function

In [4]:
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['data'],
        num_rows: 118516
    })
    validation: Dataset({
        features: ['data'],
        num_rows: 11873
    })
    test: Dataset({
        features: ['data'],
        num_rows: 11803
    })
})
{'data': {'answers': {'answer_start': [], 'text': []}, 'context': 'प्रेस्बिटेरियनिझम हा उत्तर आयर्लंडमधील सर्वात मोठा प्रोटेस्टंट संप्रदाय आहे आणि आयर्लंड बेटावर (आयर्लंडच्या अँग्लिकन चर्च नंतर) दुसरा सर्वात मोठा संप्रदाय आहे, [उद्धरण आवश्यक आहे] आणि स्कॉटिश वृक्षारोपण स्थायिकांनी अल्स्टरमध्ये आणले होते ज्यांना जेम्स व्ही द्वारे स्थलांतर करण्यास जोरदार प्रोत्साहन दिले होते. स्कॉटलंडचा, नंतर इंग्लंडचा जेम्स पहिला. अंदाजे १००,००० स्कॉटिश प्रेस्बिटेरियन्स १६०७ आणि १६९० मध्ये बॉयनच्या लढाईदरम्यान आयर्लंडच्या उत्तरेकडील काउंटीजमध्ये स्थलांतरित झाले. अल्स्टर आणि उर्वरित आयर्लंडमधील रोमन कॅथोलिकांसह प्रेस्बिटेरियन, १९व्या शतकाच्या सुरुवातीस ते मागे घेण्यापर्यंत भेदभाव करणाऱ्या दंड कायद्यांतर्गत त्रास सहन करावा लागला. आयर्लंडमधील प्रेस्

In [16]:
def preprocess_data(examples):
    questions = [item["question"] for item in examples["data"]]
    contexts = [item["context"] for item in examples["data"]]
    answers = [item["answers"] for item in examples["data"]]

    # Tokenize the inputs with truncation and padding
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    start_positions = []
    end_positions = []

    for i, ans in enumerate(answers):
        if ans["answer_start"]:
            start = ans["answer_start"][0]
            end = start + len(ans["text"][0])
            # Adjust start and end based on tokenized input
            start_positions.append(start)
            end_positions.append(end)
        else:
            start_positions.append(0)
            end_positions.append(0)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


Take subset of dataset

In [17]:
# Set a fraction of the dataset to use
fraction = 0.01  # Use 1% of the dataset

# Shuffle and select a subset of the training, validation, and test sets
train_subset = dataset["train"].shuffle(seed=42).select(range(int(len(dataset["train"]) * fraction)))
validation_subset = dataset["validation"].shuffle(seed=42).select(range(int(len(dataset["validation"]) * fraction)))
test_subset = dataset["test"].shuffle(seed=42).select(range(int(len(dataset["test"]) * fraction)))

# Create a new DatasetDict with the subsets
subset_dataset = DatasetDict({
    "train": train_subset,
    "validation": validation_subset,
    "test": test_subset
})

# Check the sizes of the subsets to confirm
print(f"Train subset size: {len(subset_dataset['train'])}")
print(f"Validation subset size: {len(subset_dataset['validation'])}")
print(f"Test subset size: {len(subset_dataset['test'])}")

Train subset size: 1185
Validation subset size: 118
Test subset size: 118


In [18]:
# Proceed with tokenization and training using subset_dataset
tokenized_subset = subset_dataset.map(preprocess_data, batched=True)







Map: 100%|██████████| 1185/1185 [00:01<00:00, 893.41 examples/s]




Map: 100%|██████████| 118/118 [00:00<00:00, 756.30 examples/s]




Map: 100%|██████████| 118/118 [00:00<00:00, 700.56 examples/s]


In [19]:
print(tokenized_subset)

DatasetDict({
    train: Dataset({
        features: ['data', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1185
    })
    validation: Dataset({
        features: ['data', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 118
    })
    test: Dataset({
        features: ['data', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 118
    })
})


Evaluation parameter

In [20]:
import evaluate
import numpy as np

# Load evaluation metrics
metric = evaluate.load("squad_v2")

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    start_logits, end_logits = predictions
    start_predictions = np.argmax(start_logits, axis=1)
    end_predictions = np.argmax(end_logits, axis=1)
    return metric.compute(predictions={"id": start_predictions, "end": end_predictions}, references=references)

Training Arguments and Trainer

In [22]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./qa_subset_maharoberta_1_results",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    log_level='debug',
    logging_steps=10,
)

In [23]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_subset["train"],
    eval_dataset=tokenized_subset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Training the model

In [24]:
# Train the model
trainer.train()

  0%|          | 0/447 [3:40:09<?, ?it/s]
Currently training with a batch size of: 8
The following columns in the training set don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: data. If data are not expected by `XLMRobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,185
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 447
  Number of trainable parameters = 277,454,594
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
                                                  
                                                  

  2%|▏         | 10/447 [09:23<7:17:43, 60.10s/it]


{'loss': 3.4466, 'grad_norm': 14.770986557006836, 'learning_rate': 2.9328859060402686e-05, 'epoch': 0.07}


                                                  
                                                  

  4%|▍         | 20/447 [19:55<7:33:42, 63.75s/it]


{'loss': 3.5451, 'grad_norm': 5.7172417640686035, 'learning_rate': 2.8657718120805368e-05, 'epoch': 0.13}


                                                  
                                                  

  7%|▋         | 30/447 [29:58<7:08:24, 61.64s/it]


...

{'loss': 2.9261, 'grad_norm': 10.497357368469238, 'learning_rate': 2.1275167785234903e-05, 'epoch': 0.87}


                                                     
                                                  

 31%|███▏      | 140/447 [2:24:20<5:18:10, 62.18s/it]


{'loss': 4.2114, 'grad_norm': 4.02404260635376, 'learning_rate': 2.0604026845637585e-05, 'epoch': 0.94}


 33%|███▎      | 149/447 [2:33:15<4:27:09, 53.79s/it]The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: data. If data are not expected by `XLMRobertaForQuestionAnswering.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 118
  Batch size = 8
































KeyError: 0

Saving the model

In [ ]:
# Save the fine-tuned model
model.save_pretrained("marathi-qa-20-mahasquad")
tokenizer.save_pretrained("marathi-qa-20-mahasquad")